## Kafka Consumer with PoP (Plain Old Python)

This example demonstrates how to consume a Kafka stream with Plain old Pythhon

We use The Confluent Kafka module: 

Documentation: https://pypi.org/project/confluent-kafka/

In [ ]:
# Required import
!pip install -q confluent-kafka

## ATM Consumer Example

This example will consume the ATM kafka stream (provided the producer is running and there is something to consume!)

In [ ]:
import json
from confluent_kafka import Consumer

consumer = Consumer({'bootstrap.servers' : 'broker:29092', 'group.id' : '*'})
print("TOPICS")
topics = consumer.list_topics() # this is a python dictionary
for topic in topics.topics.keys():
    if not topic.startswith("_"): # only show non-internal topics
        print(topic)
topic = input("Enter Kafaka topic to consume:")
consumer.subscribe([topic])

try:
    while True:
        msg = consumer.poll(1.0)

        if msg is None:
            continue
        if msg.error():
            print(f"Consumer error: {msg.error()}")
            continue

        raw = msg.value().decode('utf-8')
        payload = json.loads(raw)
        print(f"Received message: {payload}")
except KeyboardInterrupt:
    consumer.close()    

## Generic Consumer Code

As you can see the boilerplate code to consume a stream is very similar the only real difference is the topic and the JSON payload on that topic.

In [ ]:
import json
from confluent_kafka import Consumer

consumer = Consumer({'bootstrap.servers' : 'broker:29092', 'group.id' : '*'})
consumer.subscribe(['atm']) # topic

try:
    while True:
        msg = consumer.poll(1.0)

        if msg is None:
            continue
        if msg.error():
            print(f"Consumer error: {msg.error()}")
            continue

        raw = msg.value().decode('utf-8')
        payload = json.loads(raw) # de-serialize JSON in the stream to a Python dictionary!
        print(f"Received message: {payload['Id']} ${payload['Amount']}")
except KeyboardInterrupt:
    consumer.close()    